In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("eComm").getOrCreate()

In [3]:
market_fact_df = spark.read.options(inferSchema='True',header='True').csv("market_fact_full.csv")
market_fact_df.show();

+--------------+--------+-------+--------+---------+---------+--------+--------------+-------+-------------+-------------------+
|Market_fact_id|  Ord_id|Prod_id| Ship_id|  Cust_id|    Sales|Discount|Order_Quantity| Profit|Shipping_Cost|Product_Base_Margin|
+--------------+--------+-------+--------+---------+---------+--------+--------------+-------+-------------+-------------------+
|             1|Ord_5446|Prod_16|SHP_7609|Cust_1818|   136.81|    0.01|            23| -30.51|          3.6|               0.56|
|             2|Ord_5406|Prod_13|SHP_7549|Cust_1818|    42.27|    0.01|            13|   4.56|         0.93|               0.54|
|             3|Ord_5446| Prod_4|SHP_7610|Cust_1818|  4701.69|     0.0|            26| 1148.9|          2.5|               0.59|
|             4|Ord_5456| Prod_6|SHP_7625|Cust_1818|  2337.89|    0.09|            43| 729.34|         14.3|               0.37|
|             5|Ord_5485|Prod_17|SHP_7664|Cust_1818|  4233.15|    0.08|            35|1219.87|   

In [4]:
prod_dim_df = spark.read.options(inferSchema='True',header='True').csv("prod_dimen.csv")
prod_dim_df.show();

+-------+----------------+--------------------+-------+
|Prod_id|Product_Category|Product_Sub_Category|Manu_Id|
+-------+----------------+--------------------+-------+
| Prod_1| OFFICE SUPPLIES|STORAGE & ORGANIZ...|   NULL|
|Prod_10|       FURNITURE|           BOOKCASES|   NULL|
|Prod_11|       FURNITURE|              TABLES|   NULL|
|Prod_12| OFFICE SUPPLIES|              LABELS|   NULL|
|Prod_13| OFFICE SUPPLIES| PENS & ART SUPPLIES|   NULL|
|Prod_14|      TECHNOLOGY|     COPIERS AND FAX|      2|
|Prod_15|       FURNITURE|  CHAIRS & CHAIRMATS|   NULL|
|Prod_16| OFFICE SUPPLIES|SCISSORS, RULERS ...|   NULL|
|Prod_17|      TECHNOLOGY|     OFFICE MACHINES|      2|
| Prod_2| OFFICE SUPPLIES|          APPLIANCES|   NULL|
| Prod_3| OFFICE SUPPLIES|BINDERS AND BINDE...|   NULL|
| Prod_4|      TECHNOLOGY|TELEPHONES AND CO...|      2|
| Prod_5|       FURNITURE|  OFFICE FURNISHINGS|   NULL|
| Prod_6| OFFICE SUPPLIES|               PAPER|   NULL|
| Prod_7| OFFICE SUPPLIES|        RUBBER BANDS| 

#Task1 : For every order, find the Cust_id, product category and sub-category along with the profit made on each of the orders.

In [18]:
market_required_df = market_fact_df.select(["Ord_id", "Prod_id", "Cust_id","Profit","Product_Base_Margin"]);
market_required_df.show();

+--------+-------+---------+-------+-------------------+
|  Ord_id|Prod_id|  Cust_id| Profit|Product_Base_Margin|
+--------+-------+---------+-------+-------------------+
|Ord_5446|Prod_16|Cust_1818| -30.51|               0.56|
|Ord_5406|Prod_13|Cust_1818|   4.56|               0.54|
|Ord_5446| Prod_4|Cust_1818| 1148.9|               0.59|
|Ord_5456| Prod_6|Cust_1818| 729.34|               0.37|
|Ord_5485|Prod_17|Cust_1818|1219.87|               0.38|
|Ord_5446| Prod_6|Cust_1818| -47.64|               0.37|
|  Ord_31|Prod_12|  Cust_26|   1.32|               0.36|
|Ord_4725| Prod_4|Cust_1641|1137.91|               0.55|
|Ord_4725|Prod_13|Cust_1641|  45.84|               0.52|
|Ord_4725| Prod_6|Cust_1641| -27.72|               0.37|
|Ord_4743| Prod_2|Cust_1641|1675.98|               0.56|
|Ord_1925| Prod_6| Cust_708|  79.34|               0.38|
|Ord_2978|Prod_16|Cust_1088|  23.12|               0.57|
|Ord_2207|Prod_11| Cust_839|-693.23|               0.78|
|Ord_2207|Prod_10| Cust_839|-31

In [6]:
product_required_df = prod_dim_df.select(["Prod_id", "Product_Category", "Product_sub_category"]);
product_required_df.show();

+-------+----------------+--------------------+
|Prod_id|Product_Category|Product_sub_category|
+-------+----------------+--------------------+
| Prod_1| OFFICE SUPPLIES|STORAGE & ORGANIZ...|
|Prod_10|       FURNITURE|           BOOKCASES|
|Prod_11|       FURNITURE|              TABLES|
|Prod_12| OFFICE SUPPLIES|              LABELS|
|Prod_13| OFFICE SUPPLIES| PENS & ART SUPPLIES|
|Prod_14|      TECHNOLOGY|     COPIERS AND FAX|
|Prod_15|       FURNITURE|  CHAIRS & CHAIRMATS|
|Prod_16| OFFICE SUPPLIES|SCISSORS, RULERS ...|
|Prod_17|      TECHNOLOGY|     OFFICE MACHINES|
| Prod_2| OFFICE SUPPLIES|          APPLIANCES|
| Prod_3| OFFICE SUPPLIES|BINDERS AND BINDE...|
| Prod_4|      TECHNOLOGY|TELEPHONES AND CO...|
| Prod_5|       FURNITURE|  OFFICE FURNISHINGS|
| Prod_6| OFFICE SUPPLIES|               PAPER|
| Prod_7| OFFICE SUPPLIES|        RUBBER BANDS|
| Prod_8|      TECHNOLOGY|COMPUTER PERIPHERALS|
| Prod_9| OFFICE SUPPLIES|           ENVELOPES|
+-------+----------------+--------------

In [19]:
market_required_df.createOrReplaceTempView("market_table");

In [8]:
product_required_df.createOrReplaceTempView("product_table");

In [9]:
task1_df = spark.sql("select market_table.Ord_id , market_table.Cust_id, market_table.Profit, product_table.Product_Category,product_table.Product_sub_category from market_table join product_table on market_table.Prod_id == product_table.Prod_id")
task1_df.show();

+--------+---------+-------+----------------+--------------------+
|  Ord_id|  Cust_id| Profit|Product_Category|Product_sub_category|
+--------+---------+-------+----------------+--------------------+
|Ord_5446|Cust_1818| -30.51| OFFICE SUPPLIES|SCISSORS, RULERS ...|
|Ord_5406|Cust_1818|   4.56| OFFICE SUPPLIES| PENS & ART SUPPLIES|
|Ord_5446|Cust_1818| 1148.9|      TECHNOLOGY|TELEPHONES AND CO...|
|Ord_5456|Cust_1818| 729.34| OFFICE SUPPLIES|               PAPER|
|Ord_5485|Cust_1818|1219.87|      TECHNOLOGY|     OFFICE MACHINES|
|Ord_5446|Cust_1818| -47.64| OFFICE SUPPLIES|               PAPER|
|  Ord_31|  Cust_26|   1.32| OFFICE SUPPLIES|              LABELS|
|Ord_4725|Cust_1641|1137.91|      TECHNOLOGY|TELEPHONES AND CO...|
|Ord_4725|Cust_1641|  45.84| OFFICE SUPPLIES| PENS & ART SUPPLIES|
|Ord_4725|Cust_1641| -27.72| OFFICE SUPPLIES|               PAPER|
|Ord_4743|Cust_1641|1675.98| OFFICE SUPPLIES|          APPLIANCES|
|Ord_1925| Cust_708|  79.34| OFFICE SUPPLIES|               PA

In [10]:
task1_df.createOrReplaceTempView("join_table");

In [11]:
## Finding the aggregated profit per order
profit_per_Order = spark.sql("select ord_id,cust_id, sum(profit) as total_profit from join_table group by ord_id, cust_id");
profit_per_Order.show();

+--------+---------+------------+
|  ord_id|  cust_id|total_profit|
+--------+---------+------------+
|Ord_5456|Cust_1818|      729.34|
|Ord_5485|Cust_1818|     1219.87|
|Ord_5446|Cust_1818|     1070.75|
|Ord_4725|Cust_1641|     1156.03|
|  Ord_31|  Cust_26|        1.32|
|Ord_5406|Cust_1818|        4.56|
|Ord_1925| Cust_708|       79.34|
|Ord_4743|Cust_1641|     1675.98|
|Ord_2207| Cust_839|    -1010.71|
|Ord_2978|Cust_1088|       23.12|
+--------+---------+------------+



In [12]:
profit_per_Order.createOrReplaceTempView("order_customer_table");

In [13]:
frequent_customers = spark.sql("select cust_id, count(ord_id) as total_orders from order_customer_table group by cust_id order by total_orders desc")
frequent_customers.show();

+---------+------------+
|  cust_id|total_orders|
+---------+------------+
|Cust_1818|           4|
|Cust_1641|           2|
| Cust_708|           1|
|Cust_1088|           1|
|  Cust_26|           1|
| Cust_839|           1|
+---------+------------+



### Graded Q1

SparkSQL
What is the command to find the prod_id count of the most frequent product that is purchased? Suppose the view you are working with is market_table_1 and the result should be in the format Prod_id | Count of products.

In [14]:
task_1 = spark.sql("select Prod_id,count(Prod_id) from market_table group by Prod_id order by count(Prod_id)  desc limit 1")
task_1.show()

+-------+--------------+
|Prod_id|count(Prod_id)|
+-------+--------------+
| Prod_6|             4|
+-------+--------------+



### Graded Q2

How do you find the sum of Product_Base_Margin (alias sum_prod) based on the unique product_id?  The result should be in the following format: Prod_id | SUM of Product_Base_Margin (sum_prod)

In [22]:
task_2 = spark.sql("select Prod_id, sum(Product_Base_Margin) AS sum_prod from market_table group by Prod_id")
task_2.show()

+-------+------------------+
|Prod_id|          sum_prod|
+-------+------------------+
| Prod_4|1.1400000000000001|
|Prod_13|              1.06|
|Prod_16|              1.13|
| Prod_2|              0.56|
|Prod_11|              0.78|
|Prod_10|              0.77|
|Prod_17|              0.38|
|Prod_12|              0.36|
| Prod_6|1.4899999999999998|
+-------+------------------+

